In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

plt.style.use('ggplot')
%matplotlib inline

## 4.1データはひとつ、モデルはたくさん

In [2]:
!cp ../3/data3a.csv data.csv

In [3]:
df = pd.read_csv('./data.csv')

In [4]:
df.head()

,y,x,f
0,6,8.31,C
1,6,9.44,C
2,6,9.50,C
3,12,9.07,C
4,10,10.16,C


* 最大対数尤度だけみて、モデルが良いといえるのかどうか

## 4.2統計モデルの当てはまりの悪さ: 逸脱度

* 逸脱度とは?  最大対数尤度に-2をかけた値

$$
D = -2 \log L
$$

In [63]:
results = sm.GLM(df['y'], sm.add_constant(df['x']), family=sm.families.Poisson()).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -235.39
Date:                Mon, 11 Jun 2018   Deviance:                       84.993
Time:                        00:48:06   Pearson chi2:                     83.8
No. Iterations:                     4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2917      0.364      3.552      0.000       0.579       2.005
x              0.0757      0.036      2.125      0.034       0.006       0.145
==============================================================================
"""

In [8]:
results.aic

474.7725015397216

In [9]:
results.deviance

84.9929964907296

残差逸脱度（residual deviance）は

$$
D - (ポアソン分布モデルで可能な最小逸脱度)
$$

ポアソン分布モデルで可能な最小逸脱度ってなんやねん  
-> フルモデルと呼ばれているモデルの逸脱度.データ数が100個のモデルなので、パラメータ100個を使って「あてはめた」モデルということ

最小逸脱度ってなんだけっけ？  
-> いまから説明

In [18]:
# ポワソン分布の実装
def poission_pmf(_y, _lambda=3.56):
    under = np.prod([i for i in range(1, _y + 1)])
    return _lambda**_y * (math.e ** - _lambda) / under    

In [30]:
from scipy.stats import poisson
np.sum(poisson.logpmf(df['y'], df['y']))

-192.88975252449595

In [37]:
# フルモデル
full_model = np.sum([np.log(poission_pmf(y, _lambda=y)) for y in df['y']])

In [38]:
full_model

-192.88975252449595

In [40]:
# フルモデルの逸脱度
full_rd = -2 * full_model

In [54]:
# これがポアソン分布で可能な最小D
full_rd

385.7795050489919

In [66]:
(-2 * results.llf) - full_rd

84.9929964907297

In [68]:
# これが残差逸脱度↑↓
results.deviance

84.9929964907296

* 残差逸脱度とは、この例では385.8を基準とするあてはまりの悪さの相対値
* 統計モデルのパラメータを増やせば残差逸脱度が小さくなるらしい。ということがわかった（どこでだろう?
  * そうか？説明変数を増やせば、マイナス値は大きくなったはずなので、確かに残差逸脱度は小さくなるかも

In [72]:
results = sm.GLM(df['y'], sm.add_constant(df['x'])['const'], family=sm.families.Poisson()).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       99
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -237.64
Date:                Mon, 11 Jun 2018   Deviance:                       89.507
Time:                        00:53:58   Pearson chi2:                     87.1
No. Iterations:                     4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0580      0.036     57.586      0.000       1.988       2.128
==============================================================================
"""

In [73]:
results.deviance, results.aic

(89.5069375695814, 477.28644261857346)

In [75]:
(-2 * results.llf), (-2 * results.llf) - full_rd

(475.28644261857346, 89.50693756958157)

AIC（やっと理解できそう

AIC = -2{ (最大対数尤度) - (最尤推定したパラメータ数) } = -2(logL - k) = D + 2k

これが市場小さいモデルが良いモデルになる

In [81]:
results = sm.GLM(df['y'], sm.add_constant(df['x']), family=sm.families.Poisson()).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -235.39
Date:                Mon, 11 Jun 2018   Deviance:                       84.993
Time:                        01:02:03   Pearson chi2:                     83.8
No. Iterations:                     4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2917      0.364      3.552      0.000       0.579       2.005
x              0.0757      0.036      2.125      0.034       0.006       0.145
==============================================================================
"""

In [84]:
# -2 (logL  - k)
-2 * (results.llf - 2)

474.7725015397216

## 4.4 AICを説明するためのまた別の例題